In [1]:
import numpy as np

In [6]:
def calc_entropy(prob_array):
    if sum(prob_array) != 1:
        return
    return -sum(i * np.log2(i) for i in prob_array)

In [15]:
# calc_entropy([0.17, 0.08, 0.13, 0.62])
# calc_entropy([0.08, 0.12, 0.8])
# calc_entropy([0.17, 0.1, 0.1, 0.63])
# calc_entropy([0.16, 0.03, 0.81])
# calc_entropy([0.02, 0.19, 0.1, 0.69])
# calc_entropy([0.06, 0.3, 0.64])
# calc_entropy([0.23, 0.08, 0.01, 0.68])
calc_entropy([0.02, 0.1, 0.88])

0.6073635558851669

### ハフマン符号を計算してみる

In [108]:
def get_min_syms(d_inv, n):
    """
    下からn番目までの記号を取り出す?
    """
    min_prob = min(d_inv)
    min_prob_syms = d_inv.pop(min_prob)
    
    rem_len = n - len(min_prob_syms)
    # オーバー
    if rem_len < 0:
        min_prob_syms.sort(reverse=True)
        return min_prob_syms[:n]
    elif rem_len == 0:
        return min_prob_syms
    else:
        # 足りない場合は再帰
        return min_prob_syms + get_min_syms(d_inv, rem_len)

def create_connection_dict(prob_d):
    prob_d_cp = prob_d.copy()
    cnct_d = {}
    next_key = max(prob_d_cp) + 1

    while len(prob_d_cp) > 1:
        prob_d_inv = {}
        # 逆引き辞書作成
        for k, v in prob_d_cp.items():
            if v in prob_d_inv:
                prob_d_inv[v].append(k)
            else:
                prob_d_inv[v] = [k]

        print(prob_d_inv)

        min_syms = get_min_syms(prob_d_inv, 2)

        prob_d_cp[next_key] = sum(v for k, v in prob_d_cp.items() if k in min_syms)
        cnct_d[next_key] = sorted(min_syms)
        
        for i in min_syms:
            prob_d_cp.pop(i)
    
        print(prob_d_cp)
        print(cnct_d)
        next_key += 1

    return cnct_d

def generate_code(cnct_d, code_d):
    """
    枝の繋がりから符号を作成
    """
    max_key = max(cnct_d)
    children = cnct_d.pop(max_key)
    
    print(cnct_d)
    
    for i, j in enumerate(children):
        if max_key in code_d:
            code_d[j] = code_d[max_key] + str(i)
        else:
            code_d[j] = str(i)
    
    if max_key in code_d:
        code_d.pop(max_key)
    print(code_d)
    
    if any(cnct_d):
        generate_code(cnct_d, code_d)

In [110]:
# prob_arr = np.array([0.2, 0.19, 0.07, 0.05, 0.49])
# prob_arr = np.array([0.05, 0.05, 0.05, 0.05, 0.8])
prob_arr = np.array([0.12, 0.01, 0.22, 0.07, 0.02, 0.56])
print(prob_arr.sum())
prob_dict = {k: v for k, v in enumerate(prob_arr)}

connect_dict = create_connection_dict(prob_dict)

print("#" * 100)

code_dict = {}
generate_code(connect_dict, code_dict)

sorted_keys = sorted(code_dict.keys())
print("answer: ", end="")
for i in sorted_keys:
    print(code_dict[i], end=" ")
print()

1.0
{0.12: [0], 0.01: [1], 0.22: [2], 0.07: [3], 0.02: [4], 0.56: [5]}
{0: 0.12, 2: 0.22, 3: 0.07, 5: 0.56, 6: 0.03}
{6: [1, 4]}
{0.12: [0], 0.22: [2], 0.07: [3], 0.56: [5], 0.03: [6]}
{0: 0.12, 2: 0.22, 5: 0.56, 7: 0.1}
{6: [1, 4], 7: [3, 6]}
{0.12: [0], 0.22: [2], 0.56: [5], 0.1: [7]}
{2: 0.22, 5: 0.56, 8: 0.22}
{6: [1, 4], 7: [3, 6], 8: [0, 7]}
{0.22: [2, 8], 0.56: [5]}
{5: 0.56, 9: 0.44}
{6: [1, 4], 7: [3, 6], 8: [0, 7], 9: [2, 8]}
{0.56: [5], 0.44: [9]}
{10: 1.0}
{6: [1, 4], 7: [3, 6], 8: [0, 7], 9: [2, 8], 10: [5, 9]}
####################################################################################################
{6: [1, 4], 7: [3, 6], 8: [0, 7], 9: [2, 8]}
{5: '0', 9: '1'}
{6: [1, 4], 7: [3, 6], 8: [0, 7]}
{5: '0', 2: '10', 8: '11'}
{6: [1, 4], 7: [3, 6]}
{5: '0', 2: '10', 0: '110', 7: '111'}
{6: [1, 4]}
{5: '0', 2: '10', 0: '110', 3: '1110', 6: '1111'}
{}
{5: '0', 2: '10', 0: '110', 3: '1110', 1: '11110', 4: '11111'}
answer: 110 11110 10 1110 11111 0 
